# Install PuLP via pip

In [19]:
!pip install pulp


# Import and Global Variables

In [34]:
from pulp import *

# Setup Constants Function
* Vals, Rows, Cols are the sequence from 1 to n
* Sectors represent the sub-sectors within a sudoku puzzle

In [72]:
def define_constants(n):
  Vals = range(1, n+1)
  Rows = range(1, n+1)
  Cols = range(1, n+1)

  SubN = int(n/3)
  Sectors = []

  for i in range(SubN):
      for j in range(SubN):
          Sectors += [
              [
                (
                    Rows[SubN*i+k],Cols[SubN*j+l]
                ) for k in range(SubN) for l in range(SubN)
              ]
            ]

  return Vals, Rows, Cols, Sectors

Vals, Rows, Cols, Sectors = define_constants(9)

In [73]:
def define_problem_and_constraints(Vals, Rows, Cols):
  problem = LpProblem("Sudoku", LpMinimize)
  problem += 0

  options = LpVariable.dicts("Options", (Vals,Rows,Cols), 0, 1, LpInteger)
  
  # Constraint: Single Value per Option
  for r in Rows:
    for c in Cols:
        problem += lpSum(
            [options[v][r][c] for v in Vals]
        ) == 1

  # Constraint: Single Value per Row, Column, and Sector
  for v in Vals:
    for r in Rows:
        problem += lpSum(
            [options[v][r][c] for c in Cols]
        ) == 1
        
    for c in Cols:
        problem += lpSum(
            [options[v][r][c] for r in Rows]
        ) == 1

    for b in Sectors:
        problem += lpSum(
            [options[v][r][c] for (r,c) in b]
        ) == 1

  return problem, options

problem, options = define_problem_and_constraints(Vals, Rows, Cols)

In [107]:
# TODO: Have this taken in multi-dimensional array and transform into constraints
def define_initial_value_constraint(problem, options):
  problem += options[5][1][1] == 1
  problem += options[6][2][1] == 1
  problem += options[8][4][1] == 1
  problem += options[4][5][1] == 1
  problem += options[7][6][1] == 1
  problem += options[3][1][2] == 1
  problem += options[9][3][2] == 1
  problem += options[6][7][2] == 1
  problem += options[8][3][3] == 1
  problem += options[1][2][4] == 1
  problem += options[8][5][4] == 1
  problem += options[4][8][4] == 1
  problem += options[7][1][5] == 1
  problem += options[9][2][5] == 1
  problem += options[6][4][5] == 1
  problem += options[2][6][5] == 1
  problem += options[1][8][5] == 1
  problem += options[8][9][5] == 1
  problem += options[5][2][6] == 1
  problem += options[3][5][6] == 1
  problem += options[9][8][6] == 1
  problem += options[2][7][7] == 1
  problem += options[6][3][8] == 1
  problem += options[8][7][8] == 1
  problem += options[7][9][8] == 1
  problem += options[3][4][9] == 1
  problem += options[1][5][9] == 1
  problem += options[6][6][9] == 1
  problem += options[5][8][9] == 1

  return problem, options

problem, options = define_initial_value_constraint(problem, options)

In [106]:
problem.solve()
print("Status: %s" % LpStatus[problem.status]) 

Status: Optimal


In [105]:
def construct_solution_matrix(Rows, Cols, Vals, options):
  soln = []
  n = len(Rows)

  for r in Rows:
      row = []

      for c in Cols:
          for v in Vals:
            if value(options[v][r][c]):
                row.append(v)
                
                if c == n:
                  soln.append(row)
                  row = []
  return soln

solution = construct_solution_matrix(Rows, Cols, Vals, options)

In [102]:
# TODO: Make this function a little nicer and give it more of soduku vibe
def pretty_print_solution(soln):
  n = len(soln)
  sub_n = len(soln)

  line = '\n' + 2*n*'-'
  prnt = ''

  for rowi, r in enumerate(soln):
    for c in r:
      prnt += str(c) + ' '
    prnt += '\n'
  print(prnt)

pretty_print_solution(solution)

5 3 4 6 7 8 9 1 2 
6 7 2 1 9 5 3 4 8 
1 9 8 3 4 2 5 6 7 
8 5 9 7 6 1 4 2 3 
4 2 6 8 5 3 7 9 1 
7 1 3 9 2 4 8 5 6 
9 6 1 5 3 7 2 8 4 
2 8 7 4 1 9 6 3 5 
3 4 5 2 8 6 1 7 9 

